In [1]:
# Pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# No warnings about setting value on copy of slice
pd.options.mode.chained_assignment = None

# Matplotlib and seaborn for visualization
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.core.pylabtools import figsize

import seaborn as sns

pd.set_option('display.max_columns', 60)

# Imputing missing values
from sklearn.impute import SimpleImputer

# Machine Learning Models
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

# Evaluating Models
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, mean_absolute_error

import itertools

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [3]:
data_df= pd.read_csv('/input/energy-star-oracle/Usecase1_Energy_Star_Score_Dataset.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/input/energy-star-oracle/Usecase1_Energy_Star_Score_Dataset.csv'

In [ ]:
data_df.shape

In [ ]:
data_df.head(5)

In [ ]:
data_df.describe()

In [ ]:
data_df.info()

In [ ]:
data_df['ENERGY STAR Score'].value_counts()

## Looks like missing data is filled as 'Not Available', replace them with nan.

In [ ]:
data_df = data_df.replace({'Not Available': np.nan})


In [ ]:
# Iterate through the columns
for col in list(data_df.columns):
    # Select columns that should be numeric
    if ('ft²' in col or 'kBtu' in col or 'Metric Tons CO2e' in col or 'kWh' in 
        col or 'therms' in col or 'gal' in col or 'Score' in col):
        # Convert the data type to float
        data_df[col] = data_df[col].astype(float)

In [ ]:
# Look at converted data types
data_df.dtypes

## Look at Percentage of Missing Values in Each Column

In [ ]:
def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns

In [ ]:
missing_values_table(data_df)


## EDA

In [ ]:
figsize(8, 8)
plt.hist(data_df['ENERGY STAR Score'].dropna(), bins = 100);
plt.xlabel('Score'); plt.ylabel('Count'); plt.title('ENERGY Star Score Distribution');

In [ ]:
data_df['ENERGY STAR Score'].value_counts().sort_values(ascending=False)


There are two modes in the data, one at 1 and another at 100. This seems like something to follow up on. Is there some shared characteristic among buildings that have a 1 or 100? The ENERGY STAR Score is a percentile score, so it seems unreasonable that so many buildings are clustered at the two extremes. Given that Energy Star Score is based on self-reported energy usage, it might not be that good of a measure. Let's take a look at another value often used for characterizing the efficiency of a building, the Energy Use Intensity which attempts to normalize the energy usage of a building by the size of a building.

In [ ]:
plt.hist(data_df['Site EUI (kBtu/ft²)'].dropna(), range = (0, 400), bins = 20);
plt.xlabel('Site EUI'); plt.ylabel('Count'); plt.title('Site EUI Distribution');

This plot is more normally distributed with no obvious skew. The Site EUI is probably a better measure of individual building energy performance than the Energy Star Score.

## Correlations with Energy Star Score


In [ ]:
data_df.corr()['ENERGY STAR Score'].sort_values()

The most significant correlations (in absolute magnitude) are between Fuel Oil Use and Energy Star Score. This makes sense: the more fuel oil a building uses, the lower the Energy Star Score. There are no significant positive linear correlations.

The issue with using Fuel Oil Use is that this column contains 99.9% missing values. Therefore, it will not be very useful in training a model because of the limited information.

## Correlations with Site EUI


In [ ]:
data_df.corr()['Weather Normalized Site EUI (kBtu/ft²)'].sort_values()


The highest correlations here are pretty obvious: Site EUI is kBtu/ft^2, so we would expect any measure of electricity usage to be linearly positively correlated with Site EUI. We could use Gas Use or Fuel Oil use to predict EUI, but by definition these should be correlated.

## Analyzing what Buildings with 1 or 100 Energy Star Score have in common


In [ ]:
suspect = data_df[(data_df['ENERGY STAR Score'] == 1.0) | (data_df['ENERGY STAR Score'] == 100.0)]


In [ ]:
suspect.head(5)


In [ ]:
plt.hist(suspect[suspect['ENERGY STAR Score'] == 1.0]['Weather Normalized Site EUI (kBtu/ft²)'].dropna(), range = (0, 400));


In [ ]:
plt.hist(suspect[suspect['ENERGY STAR Score'] == 100.0]['Weather Normalized Site EUI (kBtu/ft²)'].dropna(), range = (0, 400));


It does look like buildings with Energy Star Scores of 100 have lower Site EUI than buildings with Energy Star Scores of 1.



## Energy Star by Building Type


In [ ]:
# Create a list of buildings with more than 80 measurements
types = data_df.dropna(subset=['ENERGY STAR Score'])
types = types['Primary Property Type - Self Selected'].value_counts()
types = list(types[types.values > 80].index)

figsize(14, 10)

# Plot each building
for b_type in types:
    # Select the building type
    subset = data_df[data_df['Primary Property Type - Self Selected'] == b_type]
    
    # Density plot of Energy Star scores
    sns.kdeplot(subset['ENERGY STAR Score'].dropna(),
               label = b_type);
    
plt.xlabel('Energy Star Score'); plt.ylabel('Density'); 
plt.title('Density Plot of Energy Star Scores');
plt.legend(loc="upper right")

Office buildings tend to have much higher Energy Star Scores than hotels and senior care communities. Residence halls and non-refrigerated warehouses also have higher scores while multifamily housing has a bi-modal distribution of scores similar to the overall distribution. It would be useful to see if this holds across a larger sample size, and to get more data to figure out why some building types tend to do better.

## Site EUI by Building Type

I identified that there was a negative correlation between the Site EUI and the Energy Star Score. We can plot the Site EUI by building type to see if there is also a difference in Site EUI between building types.

In [ ]:
figsize(14, 10)

# Plot the site EUI density plot for each building type
for b_type in types:
    # Remove outliers before plotting
    subset = data_df[(data_df['Weather Normalized Site EUI (kBtu/ft²)'] < 300) & 
                     (data_df['Primary Property Type - Self Selected'] == b_type)]
    
    # Plot the site EUI on a density plot
    sns.kdeplot(subset['Weather Normalized Site EUI (kBtu/ft²)'].dropna(), 
                label = b_type);
    
plt.xlabel('Site EUI (kBtu/ft^2)'); plt.ylabel('Density'); 
plt.title('Density Plot of Site EUI');
plt.legend(loc="upper right")

This plot provides us with another conclusion: there appears to be a negative correlation between the Site EUI and the Energy Star Score based on comparing the two distributions between building types. Building types with lower Site EUI's tend to have higher Energy Star Scores. The higher the energy use intensity (which is energy use / area), the "worse" the building's energy efficiency performance. We can visualize the relationship between the Energy Star Score and the Site EUI in a scatterplot.

## Energy Star Score vs Site EUI

In [ ]:
figsize(14, 10)

# Subset to the buildings with most measurements and remove outliers
subset = data_df[(data_df['Weather Normalized Site EUI (kBtu/ft²)'] < 300) & 
             (data_df['Primary Property Type - Self Selected'].isin(types))]

# Drop the buildings without a value
subset = subset.dropna(subset=['ENERGY STAR Score', 
                               'Weather Normalized Site EUI (kBtu/ft²)'])

subset = subset.rename(columns={'Primary Property Type - Self Selected': 'Property Type'})


# Linear Plot of Energy Star Score vs EUI
sns.lmplot(x='Weather Normalized Site EUI (kBtu/ft²)', y='ENERGY STAR Score', 
           data = subset, hue = 'Property Type', 
           scatter_kws={'alpha': 0.8, 's': 32}, fit_reg=False,height= 10, 
            aspect = 1.2);

plt.title('Energy Star Score vs Site EUI', size = 24);

The plot shows the expected negative relationship between Energy Star Score and Site EUI. This relationship looks like it holds across building types. To quantify the relationship, we can calculate the Pearson Correlation Coeffiecient between the two variables. This is a measure of linear correlation which shows both the strength and the direction of the relationship. We will look at the correlation coefficient between Energy Star Scores and several measures.

## Feature Engineering

'NYC Borough, Block and Lot (BBL) self-reported' is equal to 'BBL - 10 digits', where the first digit represents the 'Borough', next five digits 'Tax Block' and last four 'Tax Lot', plus we have already a 'Borough' column in the dataset. -- Since there are only two missing values at 'BBL - 10 digits', but more on the others, I'll check if I can use it to fill in the missing information.

In [ ]:
data_df['Largest Property Use Rate'] = data_df['Largest Property Use Type - Gross Floor Area (ft²)']/data_df['Property GFA - Self-Reported (ft²)']
data_df['2nd Property Use Rate'] = data_df['2nd Largest Property Use - Gross Floor Area (ft²)']/data_df['Property GFA - Self-Reported (ft²)']
data_df['3rd Property Use Rate'] = data_df['3rd Largest Property Use Type - Gross Floor Area (ft²)']/data_df['Property GFA - Self-Reported (ft²)']
data_df['Direct GHG Emissions Rate'] = data_df['Direct GHG Emissions (Metric Tons CO2e)']/data_df['Total GHG Emissions (Metric Tons CO2e)']
data_df['BBL - 10 digits'] = data_df['BBL - 10 digits'].str.extract('(\d+)', expand=False)
data_df['Borough'] = data_df['BBL - 10 digits'].str[0]
data_df['Tax Block'] = data_df['BBL - 10 digits'].str[1:6]
data_df['Tax Lot'] = data_df['BBL - 10 digits'].str[6:10]
data_df['Postal Code'] = data_df['Postal Code'].astype(str)
data_df['Year Built'] = data_df['Year Built'].astype(str)
data_df['Borough'] = data_df['Borough'].astype(str)
data_df['Tax Block'] = data_df['Tax Block'].astype(str)
data_df['Tax Lot'] = data_df['Tax Lot'].astype(str)
data_df['Property Id'] = data_df['Property Id'].astype(str)

The Property Type colums has a lot of values, some that appears more often and others less. I have put them all together, and split into categories that I believed made a bit of sense, while also looking at their frequency.

In [ ]:
property_type = {'Multifamily Housing':'Multifamily Housing',  
            'Residence Hall/Dormitory':'Residence Hall/Dormitory',
            'Other - Lodging/Residential':'Residence Hall/Dormitory',
            'Hotel':'Hotel',
            'Adult Education':'College/University',
            'College/University':'College/University',
            'K-12 School':'College/University',
            'Library':'College/University',
            'Vocational School':'College/University',
            'Other - Education':'College/University',
            'Office':'Office',
            'Medical Office':'Office',
            'Financial Office':'Office',
            'Bank Branch':'Office',
            'Distribution Center':'Distribution Center',
            'Self-Storage Facility':'Distribution Center',
            'Wholesale Club/Supercenter':'Distribution Center',
            'Non-Refrigerated Warehouse':'Distribution Center',
            'Fast Food Restaurant':'Food Service',
            'Food Sales':'Food Service',
            'Food Service':'Food Service',
            'Restaurant':'Food Service',
            'Supermarket/Grocery Store':'Food Service',
            'Convenience Store without Gas Station':'Food Service',
            'Other - Restaurant/Bar':'Food Service',
            'Hospital (General Medical & Surgical)':'Senior Care Community',
            'Urgent Care/Clinic/Other Outpatient':'Senior Care Community',
            'Ambulatory Surgical Center':'Senior Care Community',
            'Laboratory':'Senior Care Community',
            'Pre-school/Daycare':'Senior Care Community',
            'Senior Care Community':'Senior Care Community',
            'Outpatient Rehabilitation/Physical Therapy':'Senior Care Community',
            'Retail Store':'Retail Store',
            'Repair Services (Vehicle, Shoe, Locksmith, etc.)':'Retail Store',
            'Mailing Center/Post Office':'Retail Store',
            'Automobile Dealership':'Retail Store',
            'Mailing Center/Post Office':'Retail Store',
            'Personal Services (Health/Beauty, Dry Cleaning...':'Retail Store',
            'Enclosed Mall':'Retail Store',
            'Other - Mall':'Retail Store',
            'Other - Services':'Retail Store',
            'Other - Utility':'Retail Store',
            'Bar/Nightclub':'Recreation',
            'Bowling Alley':'Recreation',
            'Fitness Center/Health Club/Gym':'Recreation',
            'Other - Recreation':'Recreation',
            'Other - Entertainment/Public Assembly':'Recreation',
            'Performing Arts':'Recreation',
            'Social/Meeting Hall':'Recreation',
            'Museum':'Recreation',
            'Worship Facility':'Recreation',
            'Other':'Other',
            'Courthouse':'Other',
            'Other - Public Services':'Other',
            'Swimming Pool':'Other',
            'Parking':'Other',
            'Refrigerated Warehouse':'Other',
            'Data Center':'Other',
            'none':'none'
              }

In [ ]:
data_df['Largest Property Use Type'] = data_df['Largest Property Use Type'].map(property_type).astype(str)
data_df['2nd Largest Property Use Type'] = data_df['2nd Largest Property Use Type'].map(property_type).astype(str)
data_df['3rd Largest Property Use Type'] = data_df['3rd Largest Property Use Type'].map(property_type).astype(str)

In [ ]:
from sklearn.preprocessing import LabelEncoder,OrdinalEncoder

le = LabelEncoder()
data_df['n_Postal Code'] = le.fit_transform(data_df['Postal Code'])
data_df['n_Parent Property Id'] = le.fit_transform(data_df['Parent Property Id'])
data_df['n_Property Id'] = le.fit_transform(data_df['Property Id'])
data_df['n_Tax Lot'] = le.fit_transform(data_df['Tax Lot'])
data_df['n_Tax Block'] = le.fit_transform(data_df['Tax Block'])
data_df['n_3rd Largest Property Use Type'] = le.fit_transform(data_df['3rd Largest Property Use Type'])
data_df['n_2nd Largest Property Use Type'] = le.fit_transform(data_df['2nd Largest Property Use Type'])
data_df['n_Primary Property Type - Self Selected'] = le.fit_transform(data_df['Primary Property Type - Self Selected'])

#oe = OrdinalEncoder() -- discretize!!
data_df['n_Year Built'] = le.fit_transform(data_df['Year Built'])

In [ ]:
# selection = ['Postal Code','Parent Property Id','Property Id','Tax Lot','Tax Block',
#         '3rd Largest Property Use Type','2nd Largest Property Use Type',
#         'Primary Property Type - Self Selected','Year Built']

# data_df = data_df.drop(selection, axis = 1)

In [ ]:
data_df.shape

In [ ]:
# List of Variables to find correlation coefficients
features = ['Primary Property Type - Self Selected',
            'Weather Normalized Site EUI (kBtu/ft²)',
            'Weather Normalized Site Electricity Intensity (kWh/ft²)',
             'Largest Property Use Type - Gross Floor Area (ft²)',
            'Natural Gas Use (kBtu)',
            'ENERGY STAR Score']

subset = data_df[features].dropna()
subset = subset[subset['Primary Property Type - Self Selected'].isin(types)]

# Rename the columns
subset.columns = ['Property Type', 'Site EUI', 
                  'Electricity Intensity', 'Floor Area',
                  'Natural Gas', 'Energy Star Score']

# Remove outliers
subset = subset[subset['Site EUI'] < 300]

In [ ]:
# Group by the building type and calculate correlations
corrs = pd.DataFrame(subset.groupby('Property Type').corr())
corrs = pd.DataFrame(corrs['Energy Star Score'])

# Format the dataframe for display
corrs = corrs.reset_index()
corrs.columns = ['Property Type', 'Variable', 'Correlation with Score']
corrs = corrs[corrs['Variable'] != 'Energy Star Score']
corrs

This shows the correlation between Energy Star Score and three different measures by building type. For all buildings we see the following relationships: Energy Star Score is strongly negatively correlated with the Electricity Intensity and the Site EUI. The strength of the natural gas correlation depends on the dataset, but natural gas usage tends to be negatively correlated with the Energy Star Score as well. The relationship between floor area and the Energy Star score is weak for all building types.

## Remove Linearly dependent Variables

In order to find independent variables which can be used to predict the Energy Star Score, we will want to remove varialbes that are highly collinear. For example, the Site EUI and Source EUI are collinear because they both measure a very similar metric, and including both in the model will not give us independent predictors. One quick way to remove collinear variables is by calculating the correlation coefficient between every column and remove those columns with a correlation greater than a certain threshold.

In [ ]:
def corr_df(x, corr_val):
    '''
    Obj: Drops features that are strongly correlated to other features.
          This lowers model complexity, and aids in generalizing the model.
    Inputs:
          df: features df (x)
          corr_val: Columns are dropped relative to the corr_val input (e.g. 0.8)
    Output: df that only includes uncorrelated features
    '''
    # Dont want to remove correlations between Energy Star Score
    y = x['ENERGY STAR Score']
    x = x.drop(columns = ['ENERGY STAR Score'])
    
    # Creates Correlation Matrix and Instantiates
    corr_matrix = x.corr()
    iters = range(len(corr_matrix.columns) - 1)
    drop_cols = []

    # Iterates through Correlation Matrix Table to find correlated columns
    for i in iters:
        for j in range(i):
            item = corr_matrix.iloc[j:(j+1), (i+1):(i+2)]
            col = item.columns
            row = item.index
            val = abs(item.values)
            if val >= corr_val:
                # Prints the correlated feature set and the corr val
                print(col.values[0], "|", row.values[0], "|", round(val[0][0], 2))
                drop_cols.append(col.values[0])

    drops = set(drop_cols)
    x = x.drop(columns = drops)
    x = x.drop(columns = ['Site EUI (kBtu/ft²)'])
    x['ENERGY STAR Score'] = y
               
    return x

In [ ]:
data_df.shape

In [ ]:
new_data = corr_df(data_df, corr_val = 0.99)


In [ ]:
new_data.shape

In [ ]:
data_df.shape

In [ ]:
new_data= data_df.copy()

## Add in Log Transformations of Variables

In [ ]:
log_data = new_data.copy()

for col in new_data.select_dtypes('number').columns:
    log_data['log_%s' % col] = np.log10(new_data[col])
    
log_data = log_data.replace({-np.inf: np.nan})

In [ ]:
for col in log_data.select_dtypes('number').columns:
    if np.any(np.isinf(log_data[col])):
        print(col)

In [ ]:
log_data.corr()['ENERGY STAR Score'].sort_values()


The log transformed variables have a greater absolute magnitude correlation with the target. This means they might be of use in a linear regression model. We can compare performance with and without these variables.

In [ ]:
def train_test_reg(df):

    
    X = df.select_dtypes('number')
    
    X['Largest Property Use Type'] = df['Largest Property Use Type']
    X['Metered Areas (Energy)'] = df['Metered Areas (Energy)']
    X['DOF Benchmarking Submission Status'] = df['DOF Benchmarking Submission Status']
    X['Borough']= df['Borough']
    X['Largest Property Use Type']= df['Largest Property Use Type']
    
    X = pd.get_dummies(X)
    
    missing_scores = X[X['ENERGY STAR Score'].isnull()]
    
    X = X.dropna(subset = ['ENERGY STAR Score'])
    
    y = X['ENERGY STAR Score']
    if 'log_ENERGY STAR Score' in list(X.columns):
#         X = X.drop(columns=['log_Longitude', 'log_ENERGY STAR Score', 'log_Property Id'])
#         missing_scores = missing_scores.drop(columns = ['log_Longitude', 'log_ENERGY STAR Score', 'log_Property Id'])
        X = X.drop(columns=['log_Longitude', 'log_ENERGY STAR Score'])
        missing_scores = missing_scores.drop(columns = ['log_Longitude', 'log_ENERGY STAR Score'])
        
#     X = X.drop(columns = ['Property Id', 'ENERGY STAR Score', 'Longitude'])
    X = X.drop(columns = [ 'ENERGY STAR Score', 'Longitude'])
#     missing_scores = missing_scores.drop(columns = ['Property Id', 'ENERGY STAR Score', 'Longitude'])
    missing_scores = missing_scores.drop(columns = ['ENERGY STAR Score', 'Longitude'])
    feature_names = list(X.columns)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)
    
    imputer = SimpleImputer(missing_values=np.nan, strategy = 'median')
    X_train = imputer.fit_transform(X_train)
    X_test = imputer.transform(X_test)
#     missing_scores = imputer.transform(missing_scores)
    
    return X_train, X_test, y_train, y_test, missing_scores, feature_names

In [ ]:
X_train, X_test, y_train, y_test, missing_scores, feature_names = train_test_reg(new_data)


## Regression on Energy Star Scores

We will evalute our regression predictions using mean absolute error. This is simply the average error in absolute value of our predictions from the actual value and has the advantage of being easily explainable.

First we should establish a baseline measure to beat. We can use the average Energy Star Score as a baseline by predicting the mean absolute error if we simply guess the average Energy Star Score in the training date for every observation in the testing data.

In [ ]:
reg_baseline = np.mean(y_train)
reg_mae = np.mean(abs(reg_baseline - y_test))

print('Baseline Mean Absolute Error: {:0.4f}.'.format(reg_mae))

If our model cannot achieve lower than this score, then perhaps a regression approach will not work for this problem. First, let's see if a simple linear regression will work for this problem.

### Linear Regression without Log Transformed Variables


In [ ]:
lin_reg = LinearRegression()

lin_reg.fit(X_train, y_train)

lin_reg_pred = lin_reg.predict(X_test)
lin_reg_mse = np.mean(abs(lin_reg_pred - y_test))
print('Linear Regression Mean Absolute Error: {:0.4f}.'.format(lin_reg_mse))

### Linear Regression with Log Transformed Variables


In [ ]:
X_train, X_test, y_train, y_test, missing_scores, feature_names = train_test_reg(log_data)

lin_reg.fit(X_train, y_train)

lin_reg_pred = lin_reg.predict(X_test)
lin_reg_mse = np.mean(abs(lin_reg_pred - y_test))
print('Linear Regression Mean Absolute Error: {:0.4f}.'.format(lin_reg_mse))

The linear regression does not perform much better than the baseline although the log transformed features do reduce the error. Even though this is better than the baseline, let's implement a random forest regression and check the error.

### Random Forest with Log Transformed Variables


In [ ]:
rf_reg = RandomForestRegressor(n_estimators=200)
rf_reg.fit(X_train, y_train)

rf_reg_pred = rf_reg.predict(X_test)

rf_reg_mse = np.mean(abs(rf_reg_pred - y_test))
print('Random Forest Regresion Mean Absolute Error: {:0.4f}.'.format(rf_reg_mse))

### Random Forest without Log Transformed Variables


In [ ]:
X_train, X_test, y_train, y_test, missing_scores, feature_names = train_test_reg(new_data)


In [ ]:
rf_reg = RandomForestRegressor(n_estimators=200)
rf_reg.fit(X_train, y_train)

rf_reg_pred = rf_reg.predict(X_test)

rf_reg_mse = np.mean(abs(rf_reg_pred - y_test))
print('Random Forest Regresion Mean Absolute Error: {:0.4f}.'.format(rf_reg_mse))

These scores are much better and indicate that machine learning may indeed be appropriate for this problem. However, what we gain in accuracy with a more complex model, we lose in interpretability. The random forest predictions are much harder to explain than those from a linear regression. We will use the non-log transformed variables to reduce the number of features in our model.

In [ ]:
len(feature_names)

### Boosting model without Log Transformed Variables


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error


gradient_boosted=GradientBoostingRegressor(loss='absolute_error', max_depth=5,  
                          max_features=None,
                          min_samples_leaf=6,
                          min_samples_split=6,
                          n_estimators=500)

gradient_boosted.fit(X_train, y_train)
predictions = gradient_boosted.predict(X_test)
mae = mean_absolute_error(y_test, predictions)

print('Gradient Boosted Performance on the test set: MAE = %0.4f' % mae)

In [ ]:
def interpret_features(feature_list, importances):
    feature_df = pd.DataFrame({'feature': feature_list, 'importance': importances}).sort_values('importance', ascending = False)
    figsize(8, 8)
    plt.bar(feature_df[feature_df['importance'] > 0.001]['feature'], feature_df[feature_df['importance'] > 0.001]['importance'])
    plt.xticks(rotation = 90)
    plt.xlabel('feature'); plt.ylabel('importance'); plt.title('Top Feature Importances')
    plt.show();
    
    return feature_df

In [ ]:
feature_df = interpret_features(feature_names, rf_reg.feature_importances_)


In [ ]:
feature_df = interpret_features(feature_names, gradient_boosted.feature_importances_)


In [ ]:
feature_df.head(10)


According to the random forest and boosting model, the most importance features for predicting the Energy Star Score of a building are the Site EUI, the Water Intensity, Largest Property Use Type_Office, the Water Use Use. These are in line with the variables that we saw are the most correlated with the Energy Star Score.

## Conclusions

* The Site Energy Use Intensity, the Water/Electricity Intensity, the Water Use/natural gas use, and the Building Type are the most useful measures for determining the energy star score.
* Energy Star Scores are skewed high with a disproportionate global maximum at 100 and a secondary maximum at 1.
* Offices, residence halls, and non-refrigerated warehouses have higher energy star scores than senior care communities and hotels with scores of multi-family housing falling in between.
* The Site Energy Use Intensity, the Electricity Intensity, and the natural gas usage are all negatively correlated with the Energy Star Score.
* Random forest and Gradient Boosted regressors trained on the training data was able to achieve best average absolute error of between 8 to 9 percent on test set, which is significantly better than the baseline measure.
* 'Site EUI (kBtu/ft²)'=('Fuel Oil #1 Use (kBtu)'+'Fuel Oil #2 Use (kBtu)'+'Fuel Oil #4 Use (kBtu)'+ 'Fuel Oil #5 & 6 Use (kBtu)'+'Diesel #2 Use (kBtu)'+'Natural Gas Use (kBtu)'+ 'District Steam Use (kBtu)'+'Electricity Use - Grid Purchase (kBtu)')/ GFA

In [ ]:
## Hyper parameter fine-tuning

In [ ]:
# hyperparameters
# Number of trees used in the boosting process
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [100, 200, 400 ]
# Maximum depth of each tree
max_depth = [2, 4, 5, 8]
# Minimum number of samples per leaf
min_samples_leaf = [1, 2, 4, 6, 8]
# Minimum number of samples to split a node
min_samples_split = [2, 4, 6, 8, 10]
# Maximum number of features to consider for making splits
max_features = ['auto', 'sqrt', 'log2', None]
# Define the grid of hyperparameters to search
hyperparameter_grid = {'n_estimators': n_estimators,
                       'max_depth': max_depth,
                       'min_samples_leaf': min_samples_leaf,
                       'min_samples_split': min_samples_split,
                       'max_features': max_features}
# Create the model to use for hyperparameter tuning
model = GradientBoostingRegressor(random_state = 42)
# Set up the random search with 2-fold cross validation
random_cv = RandomizedSearchCV(estimator=model,
                               param_distributions=hyperparameter_grid,
                               cv=2, n_iter=10,
                               scoring = 'neg_mean_absolute_error',
                               n_jobs = -1, verbose = 1,
                               return_train_score = True,
                               random_state=42)
# Fit on the training data
random_cv.fit(X_train, y_train)

In [ ]:
random_cv.best_params_

In [ ]:
feature_names